In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
from torchtext import data
import spacy
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
os.chdir("/content/drive/My Drive")

In [3]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [4]:
fields = [('text',TEXT), ('label',LABEL)]
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = "/content/drive/My Drive",
                                        train = 'train.csv',
                                        validation = "valid.csv",
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

In [5]:
print(vars(train_data[1]))

{'text': ['i', 'just', 'keep', 'fucking', 'up', 'i', 'broke', 'a', 'promise', 'to', 'my', 'boyfriend', 'in', 'a', 'obviously', 'failed', 'suicide', 'attempt', 'he', 'called', 'me', 'and', 'i', 'answered', 'and', 'he', 'talked', 'me', 'out', 'of', 'it', 'but', 'i', 'd', 'already', 'self', '-', 'harmed', 'and', 'thus', 'broken', 'a', 'promise', 'i', 'made', 'to', 'him', 'i', 'm', 'certain', 'he', 's', 'going', 'to', 'break', 'up', 'with', 'me', 'because', 'i', 'can', 't', 'be', 'trusted', 'i', 'wish', 'i', 'would', 've', 'ignored', 'the', 'call', 'and', 'kept', 'going', 'it', 'would', 'have', 'saved', 'so', 'many', 'people', 'the', 'misery', 'of', 'having', 'to', 'put', 'up', 'with', 'me'], 'label': '1'}


In [6]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = 'glove.6B.200d',
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)
BATCH_SIZE = 200

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    device = device)

In [7]:
import torch.nn as nn
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, text, text_lengths):
        
        # text = [sentence length, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc(hidden)
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [8]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc
    
def train(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text
        
        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator):
    prediction_list = []
    true_label= []
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            rounded_preds = torch.round(torch.sigmoid(predictions))
            prediction_list.append(rounded_preds.tolist())
            true_label.append(batch.label.tolist())
            acc = binary_accuracy(predictions, batch.label)
            epoch_acc += acc.item()
        
    return epoch_acc / len(iterator), np.array([int(i) for batch in prediction_list for i in batch]), np.array([int(i) for batch in true_label for i in batch])

In [9]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [13]:
# Define Hyperparameters
num_epochs = 8
learning_rate = 0.001

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 236
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding

model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)
# Loss and optimizer
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)
criterion = criterion.to(device)

In [14]:
t = time.time()
loss=[]
acc=[]
val_acc=[]

for epoch in range(num_epochs):
    
    train_loss, train_acc = train(model, train_iterator)
    valid_acc,_,_ = evaluate(model, valid_iterator)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Acc: {valid_acc*100:.2f}%')
    
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    
print(f'time:{time.time()-t:.3f}')

	Train Loss: 0.613 | Train Acc: 65.49%
	 Val. Acc: 76.70%
	Train Loss: 0.463 | Train Acc: 78.54%
	 Val. Acc: 76.89%
	Train Loss: 0.352 | Train Acc: 85.08%
	 Val. Acc: 81.88%
	Train Loss: 0.281 | Train Acc: 88.97%
	 Val. Acc: 83.63%
	Train Loss: 0.272 | Train Acc: 88.32%
	 Val. Acc: 84.63%
	Train Loss: 0.187 | Train Acc: 92.61%
	 Val. Acc: 82.45%
	Train Loss: 0.135 | Train Acc: 95.12%
	 Val. Acc: 84.70%
	Train Loss: 0.089 | Train Acc: 96.74%
	 Val. Acc: 85.63%
time:27.300


In [15]:
accuracy, y_pred_LSTM, true_label_LSTM = evaluate(model, test_iterator)
print("Wth LSTM, accuracy on the test set is", accuracy)

Wth LSTM, accuracy on the test set is 0.8694642707705498


In [16]:
np.save("y_pred_LSTM", y_pred_LSTM)
np.save("true_label_LSTM", true_label_LSTM)
np.save("LSTM_train_acc", acc)
np.save("LSTM_train_loss", loss)